<a href="https://colab.research.google.com/github/archiechang/study/blob/master/Data%20Analytics/Data%20100/41-50/41-50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# ノック41：データを読み込んで利用データを整形しよう

In [2]:
customer=pd.read_csv("https://raw.githubusercontent.com/archiechang/study/master/Data%20Analytics/Data%20100/41-50/customer_join.csv")
uselog_months=pd.read_csv("https://raw.githubusercontent.com/archiechang/study/master/Data%20Analytics/Data%20100/41-50/use_log_months.csv")

In [3]:
year_months=list(uselog_months["年月"].unique())
uselog=pd.DataFrame()
for i in range(1,len(year_months)):
    tmp=uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"},inplace=True)
    tmp_before=uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"},inplace=True)
    tmp=pd.merge(tmp,tmp_before,on="customer_id",how="left")
    uselog=pd.concat([uselog,tmp],ignore_index=True)
uselog.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.p

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


# ノック42：退会前月の退会顧客データを作成しよう

In [4]:
from dateutil.relativedelta import relativedelta

In [5]:
exit_customer=customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"]=None
exit_customer["end_date"]=pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i]=exit_customer["end_date"].iloc[i]-relativedelta(months=1)
exit_customer["年月"]=pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
#exit_customer["年月"]=exit_customer["年月"].dt.strftime("%Y%m")
uselog["年月"]=uselog["年月"].astype(str)
exit_uselog=pd.merge(uselog,exit_customer,on=["customer_id","年月"],how="left")
print(len(uselog))
exit_uselog.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

33851


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
exit_uselog=exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,1.0,ナイト,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


# ノック43：継続顧客のデータを作成しよう

In [7]:
conti_customer=customer.loc[customer["is_deleted"]==0]
conti_uselog=pd.merge(uselog,conti_customer,on="customer_id",how="left")
print(len(conti_uselog))
conti_uselog=conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


## データのアンダーサンプリングする

In [8]:
conti_uselog=conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog=conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201901,OA116646,9,10.0,XXXXXX,C01,F,2018-12-15,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,8.250000,8.5,10.0,6.0,1.0,2019-04-30,4.0
1,201812,PL191813,5,6.0,XXXXX,C03,M,2017-08-01,NaN,CA2,0.0,ナイト,6000.0,入会費半額,5.500000,5.5,9.0,2.0,1.0,2019-04-30,20.0
2,201903,GD559311,10,5.0,XXXXX,C02,F,2018-12-01,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.000000,8.5,10.0,5.0,1.0,2019-04-30,4.0
3,201811,PL888478,3,2.0,XXXXX,C03,M,2016-05-01,NaN,CA1,0.0,ナイト,6000.0,通常,3.916667,4.0,6.0,2.0,1.0,2019-04-30,35.0
4,201806,IK871588,7,6.0,XXXX,C02,F,2016-03-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,4.5,8.0,3.0,1.0,2019-04-30,37.0


In [9]:
predict_data=pd.concat([conti_uselog,exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,OA116646,9,10.0,XXXXXX,C01,F,2018-12-15,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,8.250000,8.5,10.0,6.0,1.0,2019-04-30,4.0,NaN
1,201812,PL191813,5,6.0,XXXXX,C03,M,2017-08-01,NaN,CA2,0.0,ナイト,6000.0,入会費半額,5.500000,5.5,9.0,2.0,1.0,2019-04-30,20.0,NaN
2,201903,GD559311,10,5.0,XXXXX,C02,F,2018-12-01,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.000000,8.5,10.0,5.0,1.0,2019-04-30,4.0,NaN
3,201811,PL888478,3,2.0,XXXXX,C03,M,2016-05-01,NaN,CA1,0.0,ナイト,6000.0,通常,3.916667,4.0,6.0,2.0,1.0,2019-04-30,35.0,NaN
4,201806,IK871588,7,6.0,XXXX,C02,F,2016-03-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,4.5,8.0,3.0,1.0,2019-04-30,37.0,NaN


# ノック44：予測する月の在籍期間を作成しよう

In [10]:
predict_data["period"]=0
predict_data["now_date"]=pd.to_datetime(predict_data["年月"],format="%Y%m")
predict_data["start_date"]=pd.to_datetime(predict_data["start_date"])

for i in range(len(predict_data)):
    delta=relativedelta(predict_data["now_date"][i],predict_data["start_date"][i])
    predict_data["period"][i]=int(delta.years*12+delta.months)
predict_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,OA116646,9,10.0,XXXXXX,C01,F,2018-12-15,NaN,CA3,0.0,オールタイム,10500.0,入会費無料,8.250000,8.5,10.0,6.0,1.0,2019-04-30,4.0,NaN,0,2019-01-01
1,201812,PL191813,5,6.0,XXXXX,C03,M,2017-08-01,NaN,CA2,0.0,ナイト,6000.0,入会費半額,5.500000,5.5,9.0,2.0,1.0,2019-04-30,20.0,NaN,16,2018-12-01
2,201903,GD559311,10,5.0,XXXXX,C02,F,2018-12-01,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.000000,8.5,10.0,5.0,1.0,2019-04-30,4.0,NaN,3,2019-03-01
3,201811,PL888478,3,2.0,XXXXX,C03,M,2016-05-01,NaN,CA1,0.0,ナイト,6000.0,通常,3.916667,4.0,6.0,2.0,1.0,2019-04-30,35.0,NaN,30,2018-11-01
4,201806,IK871588,7,6.0,XXXX,C02,F,2016-03-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.916667,4.5,8.0,3.0,1.0,2019-04-30,37.0,NaN,27,2018-06-01


# ノック45：欠損値を除去しよう

In [11]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               262
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [12]:
#count_1の欠損値を除去
predict_data=predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2632
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2632
period                  0
now_date                0
dtype: int64

# ノック46：文字列型の変数を処理できるように整形しよう

## 1.データの絞り込み

In [13]:
target_col=["campaign_name","class_name","gender","count_1","routine_flg","period","is_deleted"]
predict_data=predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,入会費無料,オールタイム,F,10.0,1.0,0,0.0
1,入会費半額,ナイト,M,6.0,1.0,16,0.0
2,入会費無料,デイタイム,F,5.0,1.0,3,0.0
3,通常,ナイト,M,2.0,1.0,30,0.0
4,通常,デイタイム,F,6.0,1.0,27,0.0


## 2.ダミー変数化

In [14]:
predict_data=pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,10.0,1.0,0,0.0,0,1,0,1,0,0,1,0
1,6.0,1.0,16,0.0,1,0,0,0,0,1,0,1
2,5.0,1.0,3,0.0,0,1,0,0,1,0,1,0
3,2.0,1.0,30,0.0,0,0,1,0,0,1,0,1
4,6.0,1.0,27,0.0,0,0,1,0,1,0,1,0


## 3.ダミー変数整理

In [15]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,10.0,1.0,0,0.0,0,1,1,0,1
1,6.0,1.0,16,0.0,1,0,0,0,0
2,5.0,1.0,3,0.0,0,1,0,1,1
3,2.0,1.0,30,0.0,0,0,0,0,0
4,6.0,1.0,27,0.0,0,0,0,1,1


# ノック47：決定木を用いて退会予測モデルを作成してみよう

## 1.退会予測モデルを構築

In [16]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [17]:
exit=predict_data.loc[predict_data["is_deleted"]==1]
conti=predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X=pd.concat([exit,conti],ignore_index=True)
y=X["is_deleted"]
del X["is_deleted"]

In [18]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,y)

In [19]:
model=DecisionTreeClassifier(random_state=0)
model.fit(X_train,y_train)
y_test_pred=model.predict(X_test)
print(y_test_pred)

[1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1.

## 2.会員期間の追加

In [20]:
results_test=pd.DataFrame({"y_test":y_test,"y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
233,1.0,1.0
1700,0.0,0.0
1209,0.0,0.0
546,1.0,1.0
708,1.0,1.0


# ノック48：予測モデルの評価を行い、モデルのチューニングをしてみよう

## 1.学習用データと評価用データを用いた際の予測精度

In [21]:
correct=len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count=len(results_test)
score_test=correct/data_count
print(score_test)

0.9201520912547528


In [22]:
#学習データの適合率が97%になってしまい、過学習です。。。
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.9201520912547528
0.9771863117870723


## 2.過学習対策(max_depthの調整)

In [23]:
X=pd.concat([exit,conti],ignore_index=True)
y=X["is_deleted"]
del X["is_deleted"]
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,y)

model=DecisionTreeClassifier(random_state=0,max_depth=5)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.9182509505703422
0.9340937896070975


# ノック49：モデルに寄与している変数を確認してみよう

In [24]:
importance=pd.DataFrame({"feature_names":X.columns,"coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.355556
1,routine_flg,0.144316
2,period,0.495768
3,campaign_name_入会費半額,0.003874
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000000
7,gender_F,0.000486


# ノック50：顧客の退会を予測しよう

In [25]:
count_1=3
routing_flg=1
period=10
campaign_name="入会費無料"
class_name="オールタイム"
gender="M"

In [26]:
if campaign_name=="入会費半額":
    campaign_name_list=[1,0]
elif campaign_name=="入会費無料":
    campaign_name_list=[0,1]
elif campaign_name=="通常":
    campaign_name_list=[0,0]
if class_name=="オールタイム":
    class_name_list=[1,0]
elif class_name=="デイタイム":
    class_name_list=[0,1]
elif class_name=="ナイト":
    class_name_list=[0,0]
if gender=="F":
    gender_list=[1]
elif gender=="M":
    gender_list=[0]
input_data=[count_1,routing_flg,period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [27]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
